# HRV Comparison Analysis
This notebook compares two methods of recording HRV values with trend analysis including:
- Daily HRV values
- 7-day rolling baseline
- 60-day normal baseline
- Statistical comparison

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 8)

print("Libraries imported successfully!")

In [ ]:
# Load data
filepath = 'hrv_data.csv'  # Update with your file path

df = pd.read_csv(filepath)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

print(f"✓ Loaded {len(df)} days of data")
print(f"✓ Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Calculate rolling baselines
df['method1_7d'] = df['method1'].rolling(window=7, min_periods=1).mean()
df['method2_7d'] = df['method2'].rolling(window=7, min_periods=1).mean()
df['method1_60d'] = df['method1'].rolling(window=60, min_periods=1).mean()
df['method2_60d'] = df['method2'].rolling(window=60, min_periods=1).mean()

print("✓ Calculated 7-day and 60-day rolling baselines")
df.head(10)

In [ ]:
# Calculate statistics
correlation = df[['method1', 'method2']].corr().iloc[0, 1]
mean_values = (df['method1'] + df['method2']) / 2
diff_values = df['method1'] - df['method2']
mean_diff = diff_values.mean()
std_diff = diff_values.std()
loa_lower = mean_diff - 1.96 * std_diff
loa_upper = mean_diff + 1.96 * std_diff

print("="*60)
print("SUMMARY STATISTICS")
print("="*60)
print(f"\nMethod 1: {df['method1'].mean():.2f} ± {df['method1'].std():.2f} ms")
print(f"Method 2: {df['method2'].mean():.2f} ± {df['method2'].std():.2f} ms")
print(f"\nCorrelation: {correlation:.4f}")
print(f"Mean Difference: {mean_diff:.2f} ms")
print(f"Limits of Agreement: [{loa_lower:.2f}, {loa_upper:.2f}]")

# Paired t-test
t_stat, p_value = stats.ttest_rel(df['method1'].dropna(), df['method2'].dropna())
print(f"\nPaired t-test: t={t_stat:.4f}, p={p_value:.4f}")
print("="*60)

In [ ]:
# Plot 1: Daily HRV Values
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(df['date'], df['method1'], marker='o', label='Method 1', 
        linewidth=2, markersize=5, alpha=0.7)
ax.plot(df['date'], df['method2'], marker='s', label='Method 2', 
        linewidth=2, markersize=5, alpha=0.7)
ax.set_xlabel('Date', fontsize=13)
ax.set_ylabel('HRV (ms)', fontsize=13)
ax.set_title('Daily HRV Values - Method Comparison', fontsize=16, fontweight='bold')
ax.legend(loc='best', fontsize=12)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 2: 7-Day Rolling Baseline
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(df['date'], df['method1_7d'], label='Method 1 (7-day)', 
        linewidth=3, alpha=0.8)
ax.plot(df['date'], df['method2_7d'], label='Method 2 (7-day)', 
        linewidth=3, alpha=0.8)
ax.set_xlabel('Date', fontsize=13)
ax.set_ylabel('HRV (ms)', fontsize=13)
ax.set_title('7-Day Rolling Baseline Comparison', fontsize=16, fontweight='bold')
ax.legend(loc='best', fontsize=12)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 3: 60-Day Normal Baseline
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(df['date'], df['method1_60d'], label='Method 1 (60-day)', 
        linewidth=3.5, alpha=0.85)
ax.plot(df['date'], df['method2_60d'], label='Method 2 (60-day)', 
        linewidth=3.5, alpha=0.85)
ax.set_xlabel('Date', fontsize=13)
ax.set_ylabel('HRV (ms)', fontsize=13)
ax.set_title('60-Day Normal Baseline Comparison', fontsize=16, fontweight='bold')
ax.legend(loc='best', fontsize=12)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 4: All Trends for Method 1
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# Method 1 trends
ax1.plot(df['date'], df['method1'], 'o-', label='Daily', 
         alpha=0.4, markersize=4, linewidth=1)
ax1.plot(df['date'], df['method1_7d'], label='7-day baseline', 
         linewidth=2.5, alpha=0.85)
ax1.plot(df['date'], df['method1_60d'], label='60-day baseline', 
         linewidth=3, alpha=0.9)
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('HRV (ms)', fontsize=12)
ax1.set_title('Method 1 - All Trends', fontsize=14, fontweight='bold')
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Method 2 trends
ax2.plot(df['date'], df['method2'], 's-', label='Daily', 
         alpha=0.4, markersize=4, linewidth=1)
ax2.plot(df['date'], df['method2_7d'], label='7-day baseline', 
         linewidth=2.5, alpha=0.85)
ax2.plot(df['date'], df['method2_60d'], label='60-day baseline', 
         linewidth=3, alpha=0.9)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('HRV (ms)', fontsize=12)
ax2.set_title('Method 2 - All Trends', fontsize=14, fontweight='bold')
ax2.legend(loc='best', fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Plot 5: Correlation and Bland-Altman
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot with correlation
ax1.scatter(df['method1'], df['method2'], alpha=0.6, s=60)
z = np.polyfit(df['method1'].dropna(), df['method2'].dropna(), 1)
p = np.poly1d(z)
x_line = np.linspace(df['method1'].min(), df['method1'].max(), 100)
ax1.plot(x_line, p(x_line), "r--", linewidth=2, label='Regression line')
min_val = min(df['method1'].min(), df['method2'].min())
max_val = max(df['method1'].max(), df['method2'].max())
ax1.plot([min_val, max_val], [min_val, max_val], 'k--', 
         linewidth=1.5, alpha=0.5, label='Identity line')
ax1.set_xlabel('Method 1 HRV (ms)', fontsize=12)
ax1.set_ylabel('Method 2 HRV (ms)', fontsize=12)
ax1.set_title(f'Correlation Plot (r = {correlation:.3f})', fontsize=14, fontweight='bold')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Bland-Altman plot
ax2.scatter(mean_values, diff_values, alpha=0.6, s=60)
ax2.axhline(mean_diff, color='red', linestyle='-', linewidth=2, 
            label=f'Mean diff: {mean_diff:.2f}')
ax2.axhline(loa_lower, color='red', linestyle='--', linewidth=1.5, 
            label=f'Lower LoA: {loa_lower:.2f}')
ax2.axhline(loa_upper, color='red', linestyle='--', linewidth=1.5, 
            label=f'Upper LoA: {loa_upper:.2f}')
ax2.axhline(0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
ax2.set_xlabel('Mean of Methods (ms)', fontsize=12)
ax2.set_ylabel('Difference (Method 1 - Method 2) (ms)', fontsize=12)
ax2.set_title('Bland-Altman Plot', fontsize=14, fontweight='bold')
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Save processed data
output_file = 'hrv_comparison_processed.csv'
df.to_csv(output_file, index=False)
print(f"✓ Processed data saved to '{output_file}'")